In [1]:
import numpy as np
import pandas as pd
from math import sqrt
import scipy.stats as stats

In [2]:
def skew(ser):
    mean = np.mean(ser)
    sdev = np.std(ser)
    sumdev = 0
    for i in range(len(ser)):
        diff = ser[i]-mean
        sumdev = sumdev + (diff**3)
    num = sumdev/(sdev**3)
    skew = num/(len(ser)-1)
    return skew

In [3]:
def kurt(ser):
    mean = np.mean(ser)
    sdev = np.std(ser)
    sumdev = 0
    for i in range(len(ser)):
        diff = ser[i]-mean
        sumdev = sumdev + (diff**4)
    num = sumdev/(sdev**4)
    kurt = num/(len(ser)-1)
    return kurt

In [4]:
def ann_ret(ser,N):
    n=len(ser)
    eyp=float(ser[n-1])
    byp=float(ser[0])
    ror=((eyp-byp)/byp)
    ann_ret = ((1+ror)**(1/N))-1
    return ann_ret*100

In [5]:
def ann_vol(ser):
    sdev = np.std(ser)
    ann_vol = sdev*sqrt(12)
    return ann_vol

In [6]:
def Sharpe_Ratio(ser):
    mean = np.mean(ser)
    sd = np.std(ser)

    SR=((mean/100)-(0.03))/(sd*sqrt(12))
    return round(SR,2)

In [7]:
def Jarque_bera(ser):
    kurt = kurt(ser)
    skew = skew(ser)
    n = float(len(ser))
    
    JB = (n/6)*((skew**2) + (1/4)*(kurt-3.0)**2)
    
    if JB>=0.0 and JB<=0.05:
        return True
    else:
        return False

In [8]:
def drawdown(ini_inv,ser):
    wealth_index = ini_inv*(1 + ser).cumprod()
    prev_peaks = wealth_index.cummax()
    drawdown = (wealth_index-prev_peaks)/prev_peaks
    wealth_index.plot()
    prev_peaks.plot()
    drawdown.plot()
    
    return pd.DataFrame({"Wealth": wealth_index,
                        "Previous Peaks": prev_peaks,
                        "Drawdown": drawdown})

In [9]:
def semi_dev(ser):
    mean = np.mean(ser)
    total = 0
    for i in range(len(ser)):
        if (ser[i]<mean):
            diff = mean-ser[i]
            total+=(diff**2)
    var = total/len(ser)
    return sqrt(var)

In [10]:
def hist_var(ser,t_period,alpha):
    ser = np.array(ser)
    ser = np.sort(ser)
    index = int((t_period*(alpha/100))-1)
    return ser[index]

In [11]:
def hist_cvar(ser,t_period,alpha):
    ser = np.array(ser)
    ser = np.sort(ser)
    index = int((t_period*(alpha/100))-1)
    neg_dev = ser[:index]
    return np.mean(neg_dev)

In [12]:
def gauss_var(ser,alpha=5):
    mean = np.mean(ser)
    sdev = np.std(ser)
    z = -1.65
    var = -(mean+(z*sdev))
    return var

In [13]:
def gauss_cvar(ser,alpha=5):
    ser = np.array(ser)
    var = gauss_var(ser)
    sum_ret = 0
    for i in range(len(ser)):
        if(ser[i]<var):
            sum_ret+=ser[i]
    cvar = sum_ret/len(ser)
    return cvar
        

In [14]:
def cf_var(ser,alpha=5):
    S = skew(ser)
    K = kurt(ser)
    z = -1.65
    term1 = (1/6)*((z**2)-1)*S
    term2 = (1/24)*((z**3)-(3*z))*(K-3)
    term3 = (1/36)*(2*(z**3)-(5*z))*(S**2)
    mean = np.mean(ser)
    sdev = np.std(ser)
    Z = z+term1+term2-term3
    var = -(mean+Z*sdev)/(1-0.05)
    return var